In [ ]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("^RUI",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("AMZN",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

In [221]:
pair = 'BTC_USDT'
timeframe = '1d'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], axis=1
).assign(Date=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).drop(['timestamp'],axis=1)[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']] .set_index('Date').sort_index()

print(odf.shape)
dfSPY=odf
odf.head(4)

(2114, 5)


,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022


In [222]:
import pandas_ta as ta
dfSPY['EMA']=ta.ema(dfSPY.Close, length=80)#sma ema
dfSPY['EMA2']=ta.ema(dfSPY.Close, length=350)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=14)
dfSPY['ATR']=ta.atr(dfSPY.High,dfSPY.Low,dfSPY.Close,14)
dfSPY['ATRP'] = dfSPY.ATR / dfSPY.Close 
#dfSPY.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(dfSPY.Close, length=14, std=2.0)
my_bbands[0:50]
dfSPY=dfSPY.join(my_bbands)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY[420:425]

,Date,Open,High,Low,Close,Volume,EMA,EMA2,RSI,ATR,ATRP,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
420,2019-09-25,8497.55,8730.00,8215.64,8430.05,60783.892258,10011.009280,7950.475948,22.201764,490.621061,0.058199,8682.288564,9931.700000,11181.111436,25.160072,-0.100943
421,2019-09-26,8430.05,8465.99,7750.00,8063.73,67930.853749,9962.928310,7951.121270,19.596139,506.718842,0.062839,8221.542389,9763.751429,11305.960468,31.590502,-0.051164
422,2019-09-27,8063.49,8265.00,7852.15,8177.91,43882.924625,9918.853784,7952.413513,22.643577,500.013925,0.061142,7904.524889,9609.169286,11313.813682,35.479537,0.080188
423,2019-09-28,8177.47,8315.00,8001.09,8198.81,34473.605165,9876.383567,7953.817482,23.217248,486.720787,0.059365,7659.232330,9456.582857,11253.933385,38.012685,0.150104
424,2019-09-29,8199.38,8229.13,7890.00,8043.82,31544.211388,9831.135084,7954.330317,21.919081,476.178588,0.059198,7426.475806,9295.283571,11164.091336,40.209806,0.165171


In [233]:
"""def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal"""

def addemasignal(df):
    emasignal = [0]*len(df)
    for i in range(0, len(df)):
        if df.EMA2[i]>df.EMA[i]:
            emasignal[i]=2
        elif df.EMA2[i]<df.EMA[i]:
            emasignal[i]=1
    df['EMASignal'] = emasignal
addemasignal(dfSPY)

In [234]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if  df.Close[i]<=df['BBL_14_2.0'][i] and df.EMASignal[i]==2:
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.Close[i]>=df['BBU_14_2.0'][i] and df.EMASignal[i]==1:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY, 0.000)

In [235]:
dfSPY[dfSPY.ordersignal!=0]

,Date,Open,High,Low,Close,Volume,EMA,EMA2,RSI,ATR,ATRP,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,EMASignal,ordersignal,pointposbreak
3,2018-08-04,7412.27,7494.81,6926.00,7009.84,36288.426010,7394.512151,8320.523722,41.663638,368.146306,0.052519,7043.896253,7835.503571,8627.110890,20.205653,-0.021511,2,7009.84,7009.84
7,2018-08-08,6720.63,6721.54,6123.00,6285.00,59550.536319,7331.594982,8284.790814,30.925473,381.074546,0.060632,6293.518283,7498.031429,8702.544574,32.128784,-0.003536,2,6285.00,6285.00
38,2018-09-08,6395.54,6478.00,6111.00,6185.05,38215.032171,6974.801344,8016.992283,36.026619,320.257358,0.051779,6214.687638,6902.337143,7589.986648,19.925121,-0.021550,2,6185.05,6185.05
71,2018-10-11,6630.21,6633.92,6205.00,6252.68,48712.400332,6728.579579,7760.504933,36.276819,194.246909,0.031066,6380.420762,6582.657143,6784.893524,6.144521,-0.315820,2,6252.68,6252.68
72,2018-10-12,6252.71,6360.00,6209.00,6298.01,29004.071419,6717.948231,7752.171629,38.642837,191.157844,0.030352,6311.701135,6558.616429,6805.531722,7.529493,-0.027724,2,6298.01,6298.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,2023-02-10,21797.83,21938.16,21451.00,21625.19,338591.942470,20180.225838,24870.475745,45.669535,748.929776,0.034632,21791.067919,23001.210714,24211.353510,10.522427,-0.068537,2,21625.19,21625.19
1675,2023-03-03,23465.32,23476.95,21971.13,22354.34,319954.197850,21522.785486,24709.947312,42.935063,914.993360,0.040931,22410.686503,23735.712857,25060.739211,11.164833,-0.021262,2,22354.34,22354.34
1681,2023-03-09,21704.37,21834.99,20042.72,20362.22,443658.285840,21572.933747,24615.335284,29.304564,857.593333,0.042117,20938.010595,22673.709286,24409.407976,15.310231,-0.165867,2,20362.22,20362.22
1682,2023-03-10,20362.21,20367.78,19549.09,20150.69,618456.467100,21537.816618,24589.895710,28.242049,854.814523,0.042421,20319.502724,22456.952143,24594.401562,19.035971,-0.039489,2,20150.69,20150.69


# Visualization

In [236]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [240]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:3200].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA2, line=dict(color='yellow', width=2), name="EMA2"),        
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], line=dict(color='blue', width=1), name="BBU")]
                # ,figsize=(12,5)
                )

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=600, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [238]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [239]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 100000
    mysize = initsize*0.5
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.02
        q=1.
        
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
            if self.trades[-1].is_long and self.data.RSI[-1]>=80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            # sl1 = self.data.Close[-1]*(1-q*self.data.ATRP[-1])
            # sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1 - q*self.data.ATR[-1]/self.data.Close[-1])
            # print('sl1=',sl1)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:          
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)        
            # sl1 = self.data.Close[-1]*(1+q*self.data.ATRP[-1])
            # sl1 = max(self.data.High[-1],self.data.High[-2])*(1 + q*self.data.ATR[-1]/self.data.Close[-1])
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=1000, margin=1/5, commission=.000)
stat = bt.run()
stat

/tmp/ipykernel_273888/2694154556.py:40: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

/tmp/ipykernel_273888/2694154556.py:40: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    1764.0
Duration                               1764.0
Exposure Time [%]                         0.0
Equity Final [$]                       1000.0
Equity Peak [$]                        1000.0
Return [%]                                0.0
Buy & Hold Return [%]               257.81529
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [219]:
bt.plot()

/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p3322', ...)